In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Imports
import ee
import pandas as pd
import gee_utils

import math

In [10]:
# Trigger the authentication flow.
ee.Authenticate()

Enter verification code: 4/1AWtgzh6VjkptUy4T_7CNL7KGxZfpg2hxvbFLNtAyxRhkLKsDC8Aqi1qXH30

Successfully saved authorization token.


In [11]:
# Initialize the library.
ee.Initialize()

In [24]:
# Export parameters
EXPORT = 'drive'
BUCKET = None

# Directory setup
LSMS_EXPORT_FOLDER = 'nga_lsms_tfrecords_raw'
LSMS_CSV_PATH = '../data/nga_lsms_clusters.csv'

In [26]:
df_lsms_eth = pd.read_csv(LSMS_CSV_PATH)

# df_lsms_eth = df_lsms_eth.head(1)
print(df_lsms_eth)

     country  consumption  density        lat       lon
0        nga     0.535630        1   9.981020  9.549433
1        nga     0.540830        1   9.537398  9.502645
2        nga     0.596685        1  12.797703  9.065058
3        nga     0.611105        1   9.083250  9.875452
4        nga     0.618506        1  11.288002  5.919781
...      ...          ...      ...        ...       ...
4971     nga    46.973923        1   4.581487  7.960590
4972     nga    48.650224        0   9.138173  7.318500
4973     nga    53.263423        0   9.064088  7.478182
4974     nga    68.399484        0   6.241538  6.936828
4975     nga    70.363489        1   5.669323  6.964301

[4976 rows x 5 columns]


In [27]:
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SW_IR1', 'SW_IR2', 'TEMP']

PROJECTION = 'EPSG:3857'
SCALE = 30              
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 25         # set to a small number (<= 50) if Google Earth Engine reports memory errors

In [22]:
def export_images(df, country, export_folder, chunk_size):
    num_chunks = int(math.ceil(len(df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)
        fc = gee_utils.create_fc(df.loc[chunk_slice, :])

        roi = fc.geometry()
        composite_img = gee_utils.create_ms_img(roi).select(MS_BANDS)

        composite_img = gee_utils.add_latlon_band(composite_img)
        composite_img = gee_utils.add_nl_band(composite_img)
        composite_img = gee_utils.add_deltatemp_band(composite_img)
                
        fname = f'{country}_{i:02d}'
        tasks[(export_folder, country, i)] = gee_utils.make_tiles(
            img=composite_img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export='drive',
            prefix=export_folder, fname=fname,
            bucket=None)
    return tasks

In [28]:
#Change country param when downloading imagery
new_task = export_images(df_lsms_eth, country='nigeria', export_folder=LSMS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)

EEException: A different Operation was already started with the given request_id.